# Create ISA-API Investigation from Study Design configuration

In this notebook I will show you how you can use a study design configuration is JSON format to generate a single-study ISA investigation and how you can then serialise it in JSON and tabular (i.e. CSV) format.

Or study design configuration consists of:
* two study arms (treatment and control)
* blood and liver samples are taken during the treatment phase
* blood samples are used to perform Metabolite Profiling with two assays: mass spectrometry and nuclear magnetic resonance (NMR).

## 1. Setup

Let's import all the required libraries

In [1]:
from time import time
import os
import json

## ISA-API related imports
from isatools.model import Investigation, Study

## ISA-API create mode related imports
from isatools.create.models import StudyDesign
from isatools.create.connectors import generate_isa_study_design_from_datascriptor_config

# serializer from ISA Investigation to JSON
from isatools.isajson import ISAJSONEncoder

# ISA-Tab serialisation
from isatools import isatab

/Users/massi/.pyenv/versions/3.7.5/envs/isa-create-notebook-3.7.5/src/isatools/isatools/create/models.py:101: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  yaml_config = yaml.load(yaml_file)


## 2. Load the Study Design JSON configuration

First of all we load the study design configurator

In [2]:
with open(os.path.abspath(os.path.join("config", "study-design-with-two-arms-datascriptor.json")), "r") as config_file:
    study_design_config = json.load(config_file)
study_design_config

{'type': 'here you can describe your type of design',
 'elements': [{'id': '#element/screen',
   'name': 'screen',
   'type': 'screen',
   'duration': 7,
   'durationUnit': 'days'},
  {'id': '#element/first_treatment',
   'name': 'first treatment',
   'type': 'chemical intervention',
   'agent': 'test drug',
   'intensity': 10,
   'intensityUnit': 'mg/day',
   'duration': 30,
   'durationUnit': 'days'},
  {'id': '#element/surgical_treatment',
   'name': 'surgical treatment',
   'type': 'surgical intervention',
   'agent': 'surgery'},
  {'id': '#element/follow-up',
   'name': 'follow-up',
   'type': 'follow-up',
   'duration': 3,
   'durationUnit': 'months'},
  {'id': '#element/control_treatment',
   'name': 'control treatment',
   'type': 'chemical intervention',
   'agent': 'placebo',
   'intensity': 10,
   'intensityUnit': 'mg/day',
   'duration': 30,
   'durationUnit': 'days'}],
 'events': [{'id': '#event/sampling_0',
   'action': 'sampling',
   'input': 'subject',
   'output': 'liv

## 3. Generate the ISA Study Design from the JSON configuration
To perform the conversion we just need to use the function `generate_isa_study_design_from_datascriptor_config` (name possibly subject to change, should we drop the "isa" and "datascriptor" qualifiers?)

In [3]:
study_design = generate_isa_study_design_from_datascriptor_config(study_design_config)
assert isinstance(study_design, StudyDesign)

{}
pv_combination: ()
count: 0, prev_node: extraction_000
count: 0, prev_node: extraction_000
{}
pv_combination: ()
count: 0, prev_node: extract_000_000
count: 1, prev_node: extract_001_000
count: 0, prev_node: labelling_000_000
count: 1, prev_node: labelling_000_001
{isatools.model.OntologyAnnotation(term='instrument', term_source=None, term_accession='', comments=[]): ['Agilent QTQF 6510'], isatools.model.OntologyAnnotation(term='injection_mode', term_source=None, term_accession='', comments=[]): ['FIA', 'LC'], isatools.model.OntologyAnnotation(term='acquisition_mode', term_source=None, term_accession='', comments=[]): ['positive mode']}
pv_combination: ('Agilent QTQF 6510', 'FIA', 'positive mode')
count: 0, prev_node: labelled_extract_000_000
count: 1, prev_node: labelled_extract_000_001
pv_combination: ('Agilent QTQF 6510', 'LC', 'positive mode')
count: 0, prev_node: labelled_extract_000_000
count: 1, prev_node: labelled_extract_000_001
count: 0, prev_node: mass_spectrometry_000_00

## 4. Generate the ISA Study from the StudyDesign and embed it into an ISA Investigation

The `StudyDesign.generate_isa_study()` method returns the complete ISA-API `Study` object.

In [4]:
start = time()
study = study_design.generate_isa_study()
end = time()
print('The generation of the study design took {:.2f} s.'.format(end - start))
assert isinstance(study, Study)
investigation = Investigation(studies=[study])

/Users/massi/.pyenv/versions/3.7.5/envs/isa-create-notebook-3.7.5/src/isatools/isatools/create/models.py:2337: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(yaml_file)


Sampling protocol is Protocol(
    name=sample collection
    protocol_type=sample_collection
    uri=
    version=
    parameters=2 ProtocolParameter objects
    components=0 OntologyAnnotation objects
    comments=0 Comment objects
)
The generation of the study design took 1.83 s.


## 5. Serialize and save the JSON representation of the generated ISA Investigation

In [5]:
start = time()
inv_json = json.dumps(investigation, cls=ISAJSONEncoder, sort_keys=True, indent=4, separators=(',', ': '))
end = time()
print('The JSON serialisation of the ISA investigation took {:.2f} s.'.format(end - start))

The JSON serialisation of the study design took 0.42 s.


In [10]:
directory = os.path.abspath(os.path.join('output'))
if not os.path.exists(directory):
    os.makedirs(directory)
with open(os.path.abspath(os.path.join('output','isa-investigation-2-arms-nmr-ms.json')), 'w') as out_fp:
    json.dump(json.loads(inv_json), out_fp)

## 6. Dump the ISA Investigation to ISA-Tab

In [11]:
start = time()
isatab.dump(investigation, os.path.abspath(os.path.join('output')))
end = time()
print('The Tab serialisation of the ISA investigation took {:.2f} s.'.format(end - start))

2020-05-06 18:26:47,803 [INFO]: model.py(graph:1544) >> Building graph for object: Study(
    identifier=
    filename=s_study_01.txt
    title=
    description=
    submission_date=
    public_release_date=
    contacts=0 Person objects
    design_descriptors=0 OntologyAnnotation objects
    publications=0 Publication objects
    factors=3 StudyFactor objects
    protocols=16 Protocol objects
    assays=5 Assay objects
    sources=18 Source objects
    samples=64 Sample objects
    process_sequence=64 Process objects
    other_material=0 Material objects
    characteristic_categories=0 OntologyAnnots
    comments=0 Comment objects
    units=0 Unit objects
)
2020-05-06 18:26:47,847 [INFO]: model.py(graph:1544) >> Building graph for object: Study(
    identifier=
    filename=s_study_01.txt
    title=
    description=
    submission_date=
    public_release_date=
    contacts=0 Person objects
    design_descriptors=0 OntologyAnnotation objects
    publications=0 Publication objects
    

AttributeError: 'str' object has no attribute 'term'

To use them on the notebook we can also dump the tables to pandas DataFrames

In [ ]:
dataframes = isatab.dump_tables_to_dataframes(investigation)